DynamoDB com LangChain para histórico de bate-papo
Você pode se beneficiar das seguintes vantagens principais ao usar o DynamoDB com o LangChain para gerenciar o histórico de bate-papo:

Experiência de usuário aprimorada – Ao usar o DynamoDB para histórico de bate-papo, seu chatbot pode oferecer uma experiência consistente e personalizada. Os usuários podem retomar as conversas de onde pararam, e o chatbot pode usar interações passadas para informar suas respostas.

Integração perfeita – O LangChain fornece uma maneira perfeita de integrar com o DynamoDB, permitindo que você armazene e recupere mensagens de bate-papo com sobrecarga mínima. Ao usar a classe DynamoDBChatMessageHistory do LangChain , você pode gerenciar automaticamente o histórico de bate-papo como parte do fluxo de conversa, permitindo que o chatbot mantenha o contexto ao longo do tempo.

Escalabilidade aprimorada – Ao construir chatbots que interagem com milhares ou até milhões de usuários, gerenciar o contexto se torna um problema não trivial. A escalabilidade do DynamoDB significa que seu chatbot pode armazenar e recuperar o histórico de conversas em tempo real, não importa quantos usuários estejam interagindo com ele simultaneamente.

In [2]:
from langchain_community.chat_message_histories import (
    DynamoDBChatMessageHistory,
)

In [3]:
import boto3

# Get the service resource.
dynamodb = boto3.resource("dynamodb", region_name="sa-east-1")

# Create the DynamoDB table.
table = dynamodb.create_table(
    TableName="SessionTable",
    KeySchema=[{"AttributeName": "SessionId", "KeyType": "HASH"}],
    AttributeDefinitions=[{"AttributeName": "SessionId", "AttributeType": "S"}],
    BillingMode="PAY_PER_REQUEST",
)

# Wait until the table exists.
table.meta.client.get_waiter("table_exists").wait(TableName="SessionTable")

# Print out some data about the table.
print(table.item_count)

0


In [4]:
history = DynamoDBChatMessageHistory(table_name="SessionTable", session_id="0")

history.add_user_message("hi!")

history.add_ai_message("whats up?")

In [5]:
history.messages

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='whats up?', additional_kwargs={}, response_metadata={})]

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | ChatOpenAI(api_key="x")

In [10]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=session_id
    ),
    input_messages_key="question",
    history_messages_key="history",
)

In [11]:
# This is where we configure the session id
config = {"configurable": {"session_id": "abc123"}}

chain_with_history.invoke({"question": "Hi! I'm bob"}, config=config)

AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 22, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d57aaa13-826b-4f7b-b521-392669328f72-0', usage_metadata={'input_tokens': 22, 'output_tokens': 10, 'total_tokens': 32, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
chain_with_history.invoke({"question": "Whats my name"}, config=config)

AIMessage(content='Your name is Bob.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 43, 'total_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3432137b-86bf-4b80-a533-a0e9002ef756-0', usage_metadata={'input_tokens': 43, 'output_tokens': 5, 'total_tokens': 48, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})